In [3]:
import cvxpy as cp
import numpy as np

# Given data
A_hat = np.random.randn(3, 3) + 1j * np.random.randn(3, 3)  # Example complex matrix
K_dB = -40
K = 10**(K_dB / 20)  # Convert dB to linear scale
phi_vals = [0, np.pi/2, np.pi, 3*np.pi/2]
psi_vals = [0, np.pi/2, np.pi, 3*np.pi/2]

# Define optimization variables (real and imaginary parts separately)
x1_real = cp.Variable(3)
x1_imag = cp.Variable(3)
x2_real = cp.Variable(3)
x2_imag = cp.Variable(3)
x3_real = cp.Variable(3)
x3_imag = cp.Variable(3)

# Convert to complex variables
x1 = x1_real + 1j * x1_imag
x2 = x2_real + 1j * x2_imag
x3 = x3_real + 1j * x3_imag

# Basis vectors
e1 = np.array([1, 0, 0])
e2 = np.array([0, 1, 0])
e3 = np.array([0, 0, 1])

# Objective function: Minimize squared error
objective = cp.Minimize(
    cp.norm(A_hat @ x1 - e1, 2)**2 + 
    cp.norm(A_hat @ x2 - e2, 2)**2 + 
    cp.norm(A_hat @ x3 - e3, 2)**2
)

# Solve for all (phi, psi) combinations
best_solution = None
best_value = np.inf

for phi in phi_vals:
    for psi in psi_vals:
        # Compute complex exponentials
        exp_phi = np.exp(1j * phi)
        exp_psi = np.exp(1j * psi)

        # Constraint: |Â (x1 + x2*exp(jϕ) + x3*exp(jψ))|^2 < K^2
        combined_signal = A_hat @ (x1 + x2 * exp_phi + x3 * exp_psi)
        
        # Reformulate magnitude squared constraint
        constraint = [cp.norm(combined_signal, 2) <= K]

        # Solve the problem
        problem = cp.Problem(objective, constraint)
        result = problem.solve(solver=cp.ECOS)  # Use an efficient convex solver

        # Store the best solution
        if result < best_value:
            best_value = result
            best_solution = {
                "x1": x1_real.value + 1j * x1_imag.value,
                "x2": x2_real.value + 1j * x2_imag.value,
                "x3": x3_real.value + 1j * x3_imag.value,
                "phi": phi,
                "psi": psi,
                "objective_value": result
            }

# Display the best result
print("Best Solution:", best_solution)


Best Solution: {'x1': array([ 0.24553803+0.12456685j,  0.04185245+0.07001458j,
       -0.53955732-0.02704409j]), 'x2': array([ 0.28913169+0.25779264j, -0.02142295+0.39958501j,
       -0.45196575+0.18087627j]), 'x3': array([0.13448742-0.04359325j, 0.32827988+0.06071628j,
       0.21086011-0.08893194j]), 'phi': 3.141592653589793, 'psi': 1.5707963267948966, 'objective_value': np.float64(0.9884863354514177)}


In [4]:
A_hat

array([[ 0.37559287-1.4429553j ,  0.93666566+1.43236778j,
        -0.86304277-0.29402886j],
       [ 1.27902577+0.62081408j, -0.88262498-1.19045967j,
        -0.07071552+0.37447337j],
       [ 1.83916443-0.27111075j,  0.42913252-0.04697928j,
         0.9849475 +0.91812748j]])